## Library import

In [1]:
import math
import string

import numpy as np
import pandas as pd
import torch
from scipy.spatial.distance import cdist
from sklearn.metrics import mean_squared_error as MSE

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import ternary
from ternary.helpers import simplex_iterator

from models import ViscNet, ViscNetVFT
from data import (viscosity_df, composition_df, test_idx, train_val_idx,
                  train_idx, val_idx, compounds)

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Input compostion

0.99100
0.00189
0.00362
0.00060
0.00030
0.00055
0.00030
0.00020
0.00051
0.00019
0.00064
0.00020

In [3]:
0.7327
0.0625
0.0959
0.0158
0.0078
0.0145
0.0159
0.0103
0.0135
0.0051
0.0203
0.0057

0.0057

In [4]:
prg_t = np.zeros((1,158))
prg_t[0,0]  = 0.99100     # SiO2           
prg_t[0,4]  = 0.00189     # Al2O3           
prg_t[0,15] = 0.00362     # Li2O           
prg_t[0,7]  = 0.00060     # MgO           
prg_t[0,12] = 0.00030     # ZnO           
prg_t[0,11] = 0.00055     # BaO           
prg_t[0,23] = 0.00030     # TiO2           
prg_t[0,2]  = 0.00020     # ZrO2           
prg_t[0,1]  = 0.00051     # P2O5           
prg_t[0,3]  = 0.00019     # Na2O           
prg_t[0,8]  = 0.00064     # K2O            
prg_t[0,6]  = 0.00020     # CaO    
print(prg_t.sum())

1.0


In [5]:
prg_v = np.zeros((1,39))
prg_v[:,29] = prg_t[0,0]       # Si            
prg_v[:,0]  = prg_t[0,4]       # Al     
prg_v[:,18] = prg_t[0,15]      # Li      
prg_v[:,19] = prg_t[0,7]       # Mg     
prg_v[:,37] = prg_t[0,12]      # Zn      
prg_v[:,3]  = prg_t[0,11]      # Ba      
prg_v[:,33] = prg_t[0,23]      # Ti      
prg_v[:,38] = prg_t[0,2]       # Zr     
prg_v[:,25] = prg_t[0,1]       # P     
prg_v[:,22] = prg_t[0,3]       # Na     
prg_v[:,16] = prg_t[0,8]       # K     
prg_v[:,6]  = prg_t[0,6]       #Ca     
print(prg_v.sum())

0.9999999999999999


## T_liquidus prediction

In [6]:
t_model = keras.models.load_model('210503_tliq_NN_model.h5')

t_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2000)              318000    
_________________________________________________________________
batch_normalization (BatchNo (None, 2000)              8000      
_________________________________________________________________
activation (Activation)      (None, 2000)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0

In [7]:
t_liquidus = t_model(prg_t)

In [8]:
t_L = t_liquidus.numpy().ravel()
t_L

array([1916.175], dtype=float32)

## Visicosity at T_liquidus

In [9]:
comp = composition_df.loc[test_idx]
df = pd.DataFrame(prg_v,columns=comp.columns)
df

Al2O3  As2O3  B2O3      BaO  BeO  Bi2O3     CaO  CdO  CoO  Cs2O  ...  \
0  0.00189    0.0   0.0  0.00055  0.0    0.0  0.0002  0.0  0.0   0.0  ...   

    SiO2  SrO  Ta2O5  TeO2    TiO2  Tl2O  WO3  Y2O3     ZnO    ZrO2  
0  0.991  0.0    0.0   0.0  0.0003   0.0  0.0   0.0  0.0003  0.0002  

[1 rows x 39 columns]

In [10]:
model_dict = {
    'ViscNet': f'files/viscnet.pt',
    'ViscNet-Huber': f'files/viscnet_huber.pt',
    'ViscNet-VFT': f'files/viscnet_vft.pt',
}

v_model = torch.load(model_dict['ViscNet'])
v_model.eval()

ViscNet(
  (hidden_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=35, out_features=192, bias=True)
      (1): Dropout(p=0.07942161101271952, inplace=False)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=192, out_features=48, bias=True)
      (1): Dropout(p=0.05371454289414608, inplace=False)
      (2): Tanh()
    )
  )
  (output_layer): Sequential(
    (0): Linear(in_features=48, out_features=3, bias=True)
    (1): Sigmoid()
  )
)

In [11]:
with torch.no_grad():
    v_t_L = v_model.viscosity_from_composition(df, t_L)

print(v_t_L)

[2.7829914]


## Glass Forming Ability (Jezica calculation)

log(RC) (K/s)

<-4 outstanding / -4 to -1 good / -1 to 2 reluctant poor / >2 extremely poor

log[n(t_L)/t_L^2) (Pa s/K^2)]

\>-1 outstaning / -1 to -4 good / -4 to -7 reluctant poor / <-7 extremely poor


In [12]:
viscosity = 10**v_t_L
jezica = np.log10(viscosity/t_L/t_L)
jezica
vitrification_class = 'None'

In [13]:
if jezica >= -1 :
    vitrification_class = 'Outstanding'
elif jezica < -1 and jezica >= -4 :
    vitrification_class = 'Good (commercial glasses)'
elif jezica < -4 and jezica >= -7 :
    vitrification_class = 'Reluctant to poor'
elif jezica < -7 :
    vitrification_class = 'Extremly poor'

In [14]:
print(jezica, vitrification_class)

[-3.781879] Good (commercial glasses)
